In [4]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp

    

def calculate_constant (mass, gravity, C_Drag, diameter,rho);
    a=np.array([0,gravity,0])
    b_const=-(C_Drag*rho)/(2*mass)
    b=np.array([b_const,b_const,b_const])


def ode_system(t, state, a, b):
    x, y, z, Vx, Vy, Vz = state
    dxdt = Vx
    dydt = Vy
    dzdt = Vz
    dVxdt = a[0] + b[0] * Vx**2
    dVydt = a[1] + b[1] * Vy**2
    dVzdt = a[2] + b[2] * Vz**2
    return [dxdt, dydt, dzdt, dVxdt, dVydt, dVzdt]

def check_collision(state, backpost_x, backpost_height_min, backpost_height_max):
    x, y = state[0], state[1]
    return backpost_x <= x <= backpost_x + 0.1 and backpost_height_min <= y <= backpost_height_max

def check_hit(Y_target,Y_tolerance,X_target,X_tolerance,Z_target,Z_tolerance,Y_pos,X_pos,Z_pos):
   if np.isclose(Y_pos,Y_target,atol=Y_tolerance) and np.isclose(Z_pos,Z_target,atol=Z_tolerance):
         X_hit=X_pos
   return X_hit

def simulate(initial_conditions,debug_simulation,backpost_x, backpost_height_min, backpost_height_max,a,b,X_target,Y_target,Z_target,X_tol,Y_tol,Z_tol):
   if debug_simulation:
       x_values = []
       y_values = []
   time = 0
   X_hit=0
   last_state = []
   Isbounce=False
   while True:
      t_span = (time, time + 0.001)  # Simulate for 0.1 seconds
      sol = solve_ivp(ode_system, t_span, initial_conditions, args=(a, b), dense_output=True)
      t_eval = np.linspace(sol.t[0], sol.t[1], 1000)  #Evaluasi 100 titik
      state_eval = sol.sol(t_eval)
      X_hit = check_hit(Y_target,Y_tol,X_target,X_tol,Z_target,Z_tol,state_eval[1],state_eval[0],state_eval[2])

      if debug_simulation:
         x_values.extend(state_eval[0])
         y_values.extend(state_eval[1])

      if np.isclose(X_hit,X_target,X_tol):
         Score=1
         break

   
      for m in range(len(state_eval[1])):

         if np.isclose(state_eval[1, m], Y_target, atol=0.02):  # Check if y is approximately 2.43
            x_hit = state_eval[0, m]  # Update x_hit with the corresponding x value
         if check_collision(state_eval[:, m], backpost_x, backpost_height_min, backpost_height_max):
            if debug_simulation:
               print(f"Collision detected at time {time:.2f}s: Vx = {state_eval[3, m]:.2f}, Vy = {state_eval[4, m]:.2f}")  # Debug output
               print(f"Last coordinates before collision: x = {state_eval[0, m]:.2f}, y = {state_eval[1, m]:.2f}")  # Print last coordinates
         Isbounce=True
         last_state=[state_eval[0,-1],state_eval[1,-1],state_eval[2,-1],state_eval[3,-1],state_eval[4,-1],state_eval[5,-1]]
         break  # Exit the loop if a collision is detected
         
      
   # Check if the projectile has hit the ground
      if state_eval[1, -1] <= 0:
         if debug_simulation:
            print("Projectile hit the ground.")
            print(f"Last coordinates before hitting the ground: x = {state_eval[0, -1]:.2f}, y = {state_eval[1, -1]:.2f}")  # Print last coordinates
         last_state=[state_eval[0,-1],state_eval[1,-1],state_eval[2,-1],state_eval[3,-1],state_eval[4,-1],state_eval[5,-1]]
         break  # Exit the main loop if the projectile hits the ground
   
   return x_hit,Isbounce,last_state,Score
    


SyntaxError: expected ':' (2594813395.py, line 10)